In [12]:
import numpy as np
import pandas as pd
import pyodbc
def load_df_to_sql_server_windows_auth(df, table_name, server, database, if_exists='replace'):
    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};' \
               f'SERVER={server};DATABASE={database};Trusted_Connection=yes;'

    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    if if_exists == 'replace':
        cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}")
        conn.commit()
        col_defs = ', '.join(f"[{col}] NVARCHAR(MAX)" for col in df.columns)
        cursor.execute(f"CREATE TABLE {table_name} ({col_defs})")
        conn.commit()

    # Replace problematic values (e.g., NaN, inf) with None
    df_clean = df.replace({np.nan: None, np.inf: None, -np.inf: None})

    placeholders = ', '.join('?' for _ in df.columns)
    columns = ', '.join(f"[{col}]" for col in df.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    for row in df_clean.itertuples(index=False, name=None):
        try:
            cursor.execute(sql, row)
        except Exception as e:
            print(f"❌ Error inserting row: {row}\n{e}")

    conn.commit()
    cursor.close()
    conn.close()
    print(f"✅ Data inserted into '{table_name}' successfully.")



In [2]:
#db setup 
server = 'ALI-PY'  # Change as needed
database = 'rahmani'

### **Loading Production Data**

In [3]:
dim_date = pd.read_excel('tables/dim_date.xlsx')
dim_downtime =  pd.read_excel('tables/dim_downtime.xlsx')
dim_event_details =  pd.read_excel('tables/dim_event_details.xlsx')
dim_group =  pd.read_excel('tables/dim_group.xlsx')
dim_leader =  pd.read_excel('tables/dim_leader.xlsx')
dim_product =  pd.read_excel('tables/dim_product.xlsx')
dim_waste =  pd.read_excel('tables/dim_wate_declare.xlsx')
dim_manifact_event = pd.read_excel('tables/dim_manifact_event.xlsx')

tables_prod = [
    ('dbo.dim_date1', dim_date),
    ('dbo.dim_downtime', dim_downtime),
    ('dbo.dim_event_details', dim_event_details),
    ('dbo.dim_group', dim_group),
    ('dbo.dim_leader', dim_leader),
    ('dbo.dim_product', dim_product),
    ('dbo.dim_waste', dim_waste),
    ('dbo.dim_manifact_event', dim_manifact_event),
]

In [4]:
for table_name, df in tables_prod:
    load_df_to_sql_server_windows_auth(df, table_name, server, database, if_exists='replace')

✅ Data inserted into 'dbo.dim_date1' successfully.
✅ Data inserted into 'dbo.dim_downtime' successfully.
✅ Data inserted into 'dbo.dim_event_details' successfully.
✅ Data inserted into 'dbo.dim_group' successfully.
✅ Data inserted into 'dbo.dim_leader' successfully.
✅ Data inserted into 'dbo.dim_product' successfully.
✅ Data inserted into 'dbo.dim_waste' successfully.
✅ Data inserted into 'dbo.dim_manifact_event' successfully.


In [5]:
fact_prod = pd.read_excel('tables/fact_prod_.xlsx')
fact_waste = pd.read_excel('tables/fact_waste_.xlsx')
fact_downtime = pd.read_excel('tables/fact_downtime_.xlsx')

tables_prod_fact = [
    ('dbo.fact_prod', fact_prod),
    ('dbo.fact_waste', fact_waste),
    ('dbo.fact_downtime', fact_downtime)
]

In [9]:
for table_name, df in tables_prod_fact:
    load_df_to_sql_server_windows_auth(df, table_name, server, database, if_exists='replace')

✅ Data inserted into 'dbo.fact_prod' successfully.
✅ Data inserted into 'dbo.fact_waste' successfully.
✅ Data inserted into 'dbo.fact_downtime' successfully.


### **Perchasing Data Loading**

In [16]:
dim_datePurchasing= pd.read_excel('table_purchasing/dim_datePurchasing.xlsx')
dim_fournisseur =  pd.read_excel('table_purchasing/dim_fournisseur.xlsx')
dim_location =  pd.read_excel('table_purchasing/dim_location.xlsx')
dim_materialPurchasing =  pd.read_excel('table_purchasing/dim_materialPuchasing.xlsx')
fact_perchasing =  pd.read_excel('table_purchasing/fact_perchasing.xlsx')


tables_prod = [
    ('dbo.dim_datePurchasing', dim_datePurchasing),
    ('dbo.dim_fournisseur', dim_fournisseur),
    ('dbo.dim_location', dim_location),
    ('dbo.dim_materialPurchasing', dim_materialPurchasing),
    ('dbo.fact_perchasing', fact_perchasing)
]

In [17]:
tables_purchasing = [
    ('dbo.dim_datePurchasing', dim_datePurchasing),
    ('dbo.dim_fournisseur', dim_fournisseur),
    ('dbo.dim_location', dim_location),
    ('dbo.dim_materialPurchasing', dim_materialPurchasing),
    ('dbo.fact_perchasing', fact_perchasing)
]

In [18]:
for table_name, df in tables_purchasing:
    load_df_to_sql_server_windows_auth(df, table_name, server, database, if_exists='replace')

✅ Data inserted into 'dbo.dim_datePurchasing' successfully.
✅ Data inserted into 'dbo.dim_fournisseur' successfully.
✅ Data inserted into 'dbo.dim_location' successfully.
✅ Data inserted into 'dbo.dim_materialPurchasing' successfully.
✅ Data inserted into 'dbo.fact_perchasing' successfully.


### **Stock data table Loading**

In [22]:
dim_dateStock = pd.read_excel('table_stock/dim_dateStock.xlsx')
dim_rawmaterialStock = pd.read_excel('table_stock/dim_rawmaterialStock.xlsx')
fact_stock = pd.read_excel('table_stock/fact_stock.xlsx')

tables_stock = [
    ('dbo.dim_dateStock', dim_dateStock),
    ('dbo.dim_rawmaterialStock',dim_rawmaterialStock),
    ('dbo.fact_stock', fact_stock)
]

In [23]:
for table_name, df in tables_stock:
    load_df_to_sql_server_windows_auth(df, table_name, server, database, if_exists='replace')

✅ Data inserted into 'dbo.dim_dateStock' successfully.
✅ Data inserted into 'dbo.dim_rawmaterialStock' successfully.
✅ Data inserted into 'dbo.fact_stock' successfully.
